In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
# Configure Matplotlib to use LaTeX for text rendering
plt.rcParams['text.usetex'] = True

# Set fonts for Matplotlib
plt.rcParams['font.serif'] = 'Palatino'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams['font.monospace'] = 'Courier'
plt.rcParams['font.size'] = 11

width_cm = 15
height_cm = 8
# Convert centimeters to inches
width_in = width_cm / 2.54
height_in = height_cm / 2.54

import seaborn as sns
sns.set_theme()
figsize= (width_in, height_in)
halffigsize = (width_in/2, height_in)


import os
from metrics.plotting_utils import save_plot, get_history_from_project, save_csv, get_dataset_name_mapping
import numpy as np
import pandas as pd 
import wandb
api = wandb.Api()
from ood_detection.config import Config
from datasets.config import DATASETS_DICT


frame_name = "thesis-zsoodd_10_runs-std"
name_mapping = get_dataset_name_mapping()
aucs = ['auc-mean', 'auc-sum', 'auc-max']

## ZOC

In [ ]:
full_zoc = get_history_from_project(frame_name)
working_zoc = full_zoc.copy()
working_zoc.drop([col for col in working_zoc.columns if col not in aucs], axis=1, inplace = True)

In [ ]:
working_zoc.mean()

## Finetuned Baseline

In [ ]:
full_fine = get_history_from_project('thesis-ood_baseline-all-full_classes-test_sets')

In [ ]:
working_fine = full_fine.copy()

In [ ]:
working_fine.drop(['log_std', 'lin_std'], axis=1, inplace=True)

In [ ]:
working_fine.mean()

## MCM

In [ ]:
def get_all_finished_runs(methods, runs):
    dfs = {}
    for method in methods:
        for run in runs:
            if method == 'mls' and run != '1.0':
                continue
            name = method +'-'+ run
            print(f"Loading {name}")
            try:
                df = prep_far_ood(name)
                assert len(df) == 132
                dfs[name] = df
            except:
                print(f"{name} not finished yet")
                continue
    return dfs
def plot_errorbar(df, value_column):
    error_corr = .5 * df[value_column + '-std']
    lower = df[value_column] - error_corr
    upper = df[value_column] + error_corr
    if value_column=='LOGISTIC':
        name= 'Fine-tuning + MSP'
    else:
        name=value_column
    #mean = df[value_column].mean().round(3)
    #std = u"\u00B1{})".format(df[value_column+'-std'].mean().round(2))
    #label = name + f' (\u03bc= {mean}' + std
    
    ax.plot(df[value_column], label = name)
    ax.plot(lower, color='tab:blue', alpha=.1)
    ax.plot(upper, color='tab:blue', alpha=.1)
    ax.fill_between(range(len(df)), lower, upper, alpha=.3)

In [ ]:
def prep_mcm(name):
    full_name = f'thesis-near-mcm-{name}'
    to_keep = ['auc-max', 'auc-max-std']
    working_df = get_history_from_project(full_name)
    working_df.drop([col for col in working_df.columns if col not in to_keep], axis=1,inplace = True)
    working_df.rename(columns = {'auc-max': name.upper(), 'auc-max-std':name.upper()+'-std'}, inplace=True)
    return working_df

In [ ]:
methods = ['mls-1.0', 'msp-0.01', 'msp-1.0','msp-100.0']
full_mcm_df = pd.concat([prep_mcm(m) for m in methods], axis=1)


In [ ]:
full_mcm_df = full_mcm_df.sort_index()

### Some Correlation & std metrics

In [ ]:
full_mcm_df[['MLS-1.0-std', 'MSP-0.01-std', 'MSP-1.0-std', 'MSP-100.0-std']].mean().mean().round(3)

In [ ]:
full_mcm_df[['MLS-1.0', 'MSP-0.01', 'MSP-1.0', 'MSP-100.0']].corr()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
plot_errorbar(full_mcm_df, 'MLS-1.0')
plot_errorbar(full_mcm_df, 'MSP-1.0')
plot_errorbar(full_mcm_df, 'MSP-0.01')


ax.hlines(0.5, xmin=-.5, xmax=[len(full_mcm_df)], colors='darkgrey', alpha=0.9, linestyles='--', lw=2, label='Uninformed Guesser')
ax.vlines([range(len(full_mcm_df))], ymin=0, ymax=full_mcm_df.max(axis=1),color='darkgrey', alpha=0.6)

ax.set_ylim(0.2,1.1)
ax.set_xticklabels([name_mapping[ind] for ind in full_mcm_df.index], rotation=45, ha='right')
ax.legend(loc = 'lower left', fontsize=7)
ax.set_xlabel('ID / OOD Dataset')
ax.set_ylabel('AUROC')
# ax.set_title('Near-OOD Detection with Zero-Shot Methods')
plt.tight_layout()
plt.show()
save_plot(fig, 'NearcomparisonMcmLines', '5_2', False)

## Now compare to ZOC and linear Baseline

In [ ]:
full_mcm_1 = get_history_from_project('thesis-near-mcm-mls-1.0')
working_mcm_1 = full_mcm_1.copy()
metric_of_interest = ['auc-max', 'auc-max-std']
working_mcm_1.drop([col for col in working_mcm_1.columns if col not in metric_of_interest], axis=1,inplace = True)
working_mcm_1.rename(columns={'auc-max': 'MCM', 'auc-max-std':'MCM-std'}, inplace=True)

In [ ]:
# get zoc
full_zoc_1 = get_history_from_project('thesis-zsoodd_10_runs-std')
working = full_zoc_1.copy()
metric_of_interest = ['auc-sum', 'auc-sum-std']
working_zoc = working[metric_of_interest].rename(columns={'auc-sum': 'ZOC', 'auc-sum-std':'ZOC-std'})

In [ ]:
# get logistic
full_log = get_history_from_project('thesis-ood_baseline-all-full_classes-test_sets')
logistic = full_log.copy()
logistic = logistic[['log_AUC', 'log_std']]
logistic.rename(columns={'log_AUC': 'LOGISTIC', 'log_std': 'LOGISTIC-std'},inplace=True)

In [ ]:
full_near = pd.concat([working_mcm_1,working_zoc, logistic ], axis=1)
full_near = full_near.sort_index()

## Some Corr and STD statistics

### stds...

In [ ]:
print("MAX STD")
full_near[[col for col in full_near.columns if "-std" in col]].max().round(3)

In [ ]:
print("MEAN STD")
full_near[[col for col in full_near.columns if "-std" in col]].mean().round(3)

In [ ]:
full_near.sort_values('ZOC-std', ascending=False)

In [ ]:
full_near[['MCM', 'ZOC', 'LOGISTIC']].corr().round(3)

In [ ]:
full_near[['MCM', 'ZOC', 'LOGISTIC', 'MCM-std', 'ZOC-std', 'LOGISTIC-std']].mean().round(3)

#### Without the shitty ones, not that bad

In [ ]:
full_near.drop(['mnist', 'svhn', 'gtsrb'])[['MCM', 'ZOC', 'LOGISTIC']].mean().round(3)

In [ ]:
def f(x,y):
    return x-y
differences = full_near.copy()
differences['ZOC-MCM'] = differences.apply(lambda x: f(x.ZOC,x.MCM), axis=1)
differences['LOGISTIC-ZOC'] = differences.apply(lambda x: f(x.LOGISTIC,x.ZOC), axis=1)


In [ ]:
differences.loc[differences['ZOC-MCM'].idxmax()]

In [ ]:
differences.loc[differences['LOGISTIC-ZOC'].idxmax()]

In [ ]:
full_near.loc[['cifar10', 'cifar100', 'imagenet']]['ZOC']

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
plot_errorbar(full_near, 'MCM')
plot_errorbar(full_near, 'ZOC')
plot_errorbar(full_near, 'LOGISTIC')

ax.hlines(0.5, xmin=-.5, xmax=[len(full_near)-1], colors='darkgrey', alpha=0.9, linestyles='--', lw=2, label='Uninformed Guesser')
ax.vlines([range(len(full_near))], ymin=0, ymax=full_near.max(axis=1),color='darkgrey', alpha=0.6)

ax.set_ylim(0.2,1.1)
ax.set_xticklabels([name_mapping[ind] for ind in full_near.index], rotation=45, ha='right')
ax.legend(loc='lower left', fontsize=7)
ax.set_xlabel('ID / OOD Dataset')
ax.set_ylabel('AUROC')
#ax.set_title('Zero-shot and fine-tuned OOD Detection')
plt.tight_layout()
plt.show()
save_plot(fig, 'nearComparisonMZL', '5_2', False)

# Get correletions

In [ ]:
from scipy import stats
def line(x, y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return x * slope + intercept 
def plot_line(x,y):
    new_y = line(x,y)
    ax.plot(x,new_y)

In [ ]:
log_class = get_history_from_project('thesis-classification-logistic')
log_class.rename(columns={'Acc': 'Logistic Accuracy'})
zsa_class = tip_df = get_history_from_project('thesis-tip-adapters-16_shots-test')['ZEROSHOT']


In [ ]:
to_plot = ['ZOC','MCM', 'LOGISTIC']
corr_df = pd.concat([full_near[to_plot], zsa_class], axis=1)
corr_df.sort_values('ZEROSHOT', inplace=True)

In [ ]:
corr_df.corr()

In [ ]:
fig, ax = plt.subplots(figsize=halffigsize)
x = corr_df['ZEROSHOT']
y1 = corr_df['MCM']
y2 = corr_df['ZOC']
y3 = corr_df['LOGISTIC']

plot_line(x,y1)
plot_line(x,y2)
# ax.scatter(x, y1, label= f"MCM (r = {corr_df.corr()['ZEROSHOT']['MCM'].round(2)}) ")
# ax.scatter(x, y2, label= f"ZOC (r = {corr_df.corr()['ZEROSHOT']['ZOC'].round(2)}) ")
ax.scatter(x, y1, label= f"MCM")
ax.scatter(x, y2, label= f"ZOC")
# ax.set_ylim(0.2,1.1)
# ax.set_xticklabels(corr_df.index, rotation=45, ha='right')
ax.legend(loc='upper left', fontsize=7)
ax.set_xlabel('Zero-Shot Accuracy')
ax.set_ylabel('AUROC')
plt.tight_layout()
#plt.title('Correlation with Zero-Shot Accuracy')
plt.show()
save_plot(fig, 'corr_with_zsa', '5_2', False)

### Distances corr

In [ ]:
clp = get_history_from_project('thesis-clp-temperature-100runs')
mmd = get_history_from_project('thesis-mmd-100')
clp_one_temp = clp[(clp['temperature'] < 1.13) & (clp['temperature'] > 0.98)]
clp_one_temp = clp_one_temp.drop(['temperature', 'std'], axis=1)
mmd = mmd.drop('std', axis=1)
distances = pd.concat([mmd, clp_one_temp], axis=1)

In [ ]:
# quick add zsa...
# get zsa... tip_df = get_history_from_project('thesis-tip-adapters-16_shots-test')
tip_df = get_history_from_project('thesis-tip-adapters-16_shots-test')
#add zsa to distances


In [ ]:
to_plot = ['ZOC','MCM', 'LOGISTIC']
distances_df = pd.concat([full_near[to_plot], distances], axis=1)
distances_df['ZSA'] = distances_df.apply(lambda x: tip_df['ZEROSHOT'][x.name], axis=1)
distances_df = distances_df.sort_values('clp')

In [ ]:
distances_df.max()

In [ ]:
distances_df.min()

In [ ]:
distances_df

In [ ]:
distances_df.corr().round(2)

In [ ]:
fig, ax = plt.subplots(figsize=halffigsize)
x = distances_df['clp']
y1 = distances_df['MCM']
y2 = distances_df['ZOC']
y3 = distances_df['LOGISTIC']

plot_line(x,y1)
plot_line(x,y2)

# the points
# ax.scatter(x, y1, label= f"MCM (r = {distances_df.corr()['clp']['MCM'].round(2)}) ")
# ax.scatter(x, y2, label= f"ZOC (r = {distances_df.corr()['clp']['ZOC'].round(2)}) ")
ax.scatter(x, y1, label= f"MCM")
ax.scatter(x, y2, label= f"ZOC")
# config stuff
# ax.set_ylim(0.2,1.1)
# ax.set_xticklabels(corr_df.index, rotation=45, ha='right')
ax.legend(loc='upper right', fontsize=7)
ax.set_xlabel('Confusion Log Probability')
ax.set_ylabel('AUROC')
plt.tight_layout()
# plt.title('Correlation with CLP')
plt.show()
save_plot(fig, 'corr_with_clp', '5_2', False)

In [ ]:
def plot_line(x,y,ax):
    new_y = line(x,y)
    ax.plot(x,new_y)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=figsize, sharey=True)
x = corr_df['ZEROSHOT']
y1 = corr_df['MCM']
y2 = corr_df['ZOC']
y3 = corr_df['LOGISTIC']

plot_line(x,y1, axs[0])
plot_line(x,y2, axs[0])
# ax.scatter(x, y1, label= f"MCM (r = {corr_df.corr()['ZEROSHOT']['MCM'].round(2)}) ")
# ax.scatter(x, y2, label= f"ZOC (r = {corr_df.corr()['ZEROSHOT']['ZOC'].round(2)}) ")
axs[0].scatter(x, y1, label= f"MCM")
axs[0].scatter(x, y2, label= f"ZOC")
# ax.set_ylim(0.2,1.1)
# ax.set_xticklabels(corr_df.index, rotation=45, ha='right')
axs[0].legend(loc='upper left', fontsize=7)
axs[0].set_xlabel('Zero-Shot Accuracy')
axs[0].set_ylabel('AUROC')

x = distances_df['clp']
y1 = distances_df['MCM']
y2 = distances_df['ZOC']
y3 = distances_df['LOGISTIC']

plot_line(x,y1, axs[1])
plot_line(x,y2, axs[1])

# the points
# ax.scatter(x, y1, label= f"MCM (r = {distances_df.corr()['clp']['MCM'].round(2)}) ")
# ax.scatter(x, y2, label= f"ZOC (r = {distances_df.corr()['clp']['ZOC'].round(2)}) ")
axs[1].scatter(x, y1, label= f"MCM")
axs[1].scatter(x, y2, label= f"ZOC")
# config stuff
# ax.set_ylim(0.2,1.1)
# ax.set_xticklabels(corr_df.index, rotation=45, ha='right')
axs[1].legend(loc='upper right', fontsize=7)
axs[1].set_xlabel('Confusion Log Probability')


plt.tight_layout()
#plt.title('Correlation with Zero-Shot Accuracy')
plt.show()
save_plot(fig, 'corr_with_zsa_and_clp', '5_2', False)